In [1]:
import os
from matplotlib import pyplot as plt
import rasterio
import numpy as np
from skimage import exposure
percent_small = 0.2 ; percent_large = 100 - percent_small ; 

path = "/mnt/hdd/eric/.tmp_ipy/00.Data/K3_20221211/_264"
dir_list = os.listdir(path)


def contrast_stretch(band, band_small, band_large):
    band_small, band_large = np.percentile(band, (percent_small, percent_large))
    scaled = exposure.rescale_intensity(band, in_range=(band_small, band_large), out_range=np.float32)
    return scaled

def normalize_img(img):
    return (img-np.min(img))/(np.max(img)-np.min(img))

In [14]:
path

'/mnt/hdd/eric/.tmp_ipy/00.Data/K3_20221211/_264'

In [16]:
dir

'K3_20221211042651_56381_09361264_L1G_PG.tif'

In [27]:

full_path = os.path.join(path,dir)
full_path.split("/")[-1].split("_")[-1] == "PG.tif"

True

In [33]:
a = os.path.join(path,dir)

In [34]:
os.path.exists(a)

True

In [37]:
tmp_ =[]
for dir in os.listdir(path):
    a_color = os.path.join(path,dir)
    
    dat_ =  rasterio.open(a_color, dtype = 'float64')
    dat_reshape = np.reshape(dat_.read(), dat_.shape)
    nor_r = normalize_img(dat_reshape)
    
    tmp_.append(nor_r)
# BGR to RGB 
# [0,1,2] => [2,1,0]
tmp_resverse = tmp_[::-1]

img_temp = np.stack(tmp_resverse, axis=-1)
    

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


ValueError: cannot reshape array of size 3546978304 into shape (26956,32896)

In [36]:
out_png_path = os.path.join(path,"_RGB.png")
plt.imsave(out_png_path, img_temp)

In [6]:

# p = 5
# dir = dir_list[p]

for dir in dir_list[:-1] :
    print(dir)
    if 'MSC' in dir :
        files = [file for file in os.listdir(f'{path}{dir}') if file.endswith('.tif')]
        red = f'{path}{dir}/{files[3]}'
        green = f'{path}{dir}/{files[0]}'
        blue = f'{path}{dir}/{files[1]}'
    else :
        red = f'{path}{dir}/{dir}_PR.tif'
        green = f'{path}{dir}/{dir}_PG.tif'
        blue = f'{path}{dir}/{dir}_PB.tif'
        
    dat_r = rasterio.open(red, dtype = 'float64')
    dat_g = rasterio.open(green, dtype = 'float64')
    dat_b = rasterio.open(blue, dtype = 'float64')
    dat_r_reshape = np.reshape(dat_r.read(), dat_r.shape)
    dat_g_reshape = np.reshape(dat_g.read(), dat_g.shape)
    dat_b_reshape = np.reshape(dat_b.read(), dat_b.shape)

    nor_r = normalize_img(dat_r_reshape)
    nor_g = normalize_img(dat_g_reshape)
    nor_b = normalize_img(dat_b_reshape)

    img_temp = np.stack((nor_r, nor_g, nor_b), axis=-1)

K3_20221211042651_56381_09361264_L1G_PG.tif


RasterioIOError: /mnt/hdd/eric/.tmp_ipy/00.Data/K3_20221211/_264K3_20221211042651_56381_09361264_L1G_PG.tif/K3_20221211042651_56381_09361264_L1G_PG.tif_PR.tif: No such file or directory

In [ ]:

    dat_r = rasterio.open(red, dtype = 'float64')
    dat_g = rasterio.open(green, dtype = 'float64')
    dat_b = rasterio.open(blue, dtype = 'float64')
    dat_r_reshape = np.reshape(dat_r.read(), dat_r.shape)
    dat_g_reshape = np.reshape(dat_g.read(), dat_g.shape)
    dat_b_reshape = np.reshape(dat_b.read(), dat_b.shape)

    nor_r = normalize_img(dat_r_reshape)
    nor_g = normalize_img(dat_g_reshape)
    nor_b = normalize_img(dat_b_reshape)

    img_temp = np.stack((nor_r, nor_g, nor_b), axis=-1)
    # plt.imshow(img_temp) ; plt.show()
    out_png_path = f'{path}{dir}/{dir}_RGB.png'
    plt.imsave(out_png_path, img_temp)
    
    img_temp = img_temp.transpose(2,0,1)
    
    # Make tiff file
    outpath = f'{path}{dir}/{dir}_RGB_ver2.1.tif'
    png_path = f'{path}{dir}/{dir}_RGB.tif'
    raster = rasterio.open(png_path)

    # Write to TIFF
    kwargs = raster.meta
    # kwargs.update(
    #     dtype=rasterio.float32,
    #     count=3,
    #     compress='lzw')

    with rasterio.open(outpath, 'w', **kwargs) as dst:
        dst.write(img_temp.astype(rasterio.float32))
    raster.close()
    dst.close()

# PNG to Tiff

from skimage.io import imshow, imread
png_path = f'{path}{dir}/{dir}_RGB_ver2.png'
# opening a singleband image
img = imread(png_path)
RGB = img[:, :, :3]
# RGB.shape
RGB_temp = RGB.transpose(2,0,1)
# RGB_temp.shape

# Make tiff file
outpath = f'{path}{dir}/{dir}_RGB_ver2.3.tif'
png_path = f'{path}{dir}/{dir}_RGB.tif'
raster = rasterio.open(png_path)

# Write to TIFF
kwargs = raster.meta
# kwargs.update(
#     dtype=rasterio.float32,
#     count=3,
#     compress='lzw')

with rasterio.open(outpath, 'w', **kwargs) as dst:
    dst.write(RGB_temp.astype(rasterio.float32))
raster.close()
dst.close()